<a href="https://colab.research.google.com/github/SaeSaeeda/ComputerScience-2022-2025/blob/main/language_technology_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#nessasary imports for the project
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import pandas as pd
from tabulate import tabulate
!pip install pandas requests beautifulsoup4

In [67]:
#creates the primary table for the project, going through each link and adding it to the series column
url = "https://en.wikipedia.org/wiki/Korean_drama"
response = requests.get(url)
#checks if information can be scraped from the website
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find_all('table', {'class': 'wikitable'})[2]

    if table:
        df = pd.read_html(str(table), header=0)[0]
        df['Network'].fillna(method='ffill', inplace=True)
        df = df.drop(columns=['Ref'])
        df.to_csv('kdramas_info.csv', index=False, encoding='utf-8')

        print("Kdramas information saved to 'kdramas_info.csv'")
    else:
        print("Table not found on the page.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Kdramas information saved to 'kdramas_info.csv'


In [68]:
#adding the column called drama link which is the hyperlink of the drama
url = "https://en.wikipedia.org/wiki/Korean_drama"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find_all('table', {'class': 'wikitable'})[2]
    if table:
        df = pd.read_html(str(table), header=0)[0]
        df['Network'].fillna(method='ffill', inplace=True)
        drama_links = []
        for cell in table.select('tr td:nth-of-type(2)'):
            link = cell.find('a')
            drama_links.append(link['href'] if link else None)
        df['Drama Link'] = drama_links
        df = df.drop(columns=['Ref'])

        df.to_csv('kdramas_info.csv', index=False, encoding='utf-8')

        print("Kdramas information saved to 'kdramas_info.csv'")
    else:
        print("Table not found on the page.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Kdramas information saved to 'kdramas_info.csv'


In [69]:
#gets the cast members of the kdrama,
def get_cast_info(series_url):
    response = requests.get(series_url)
    if response.status_code == 200:
        series_soup = BeautifulSoup(response.text, 'html.parser')
        cast_heading = series_soup.find('span', {'id': 'Cast'})
        cast_text = ''

        if cast_heading:
            parent_section = cast_heading.find_parent()
            paragraphs = []
            current_element = parent_section.find_next()

            while current_element and not current_element.name.startswith('h2'):
                if current_element.name in ['p', 'ul', 'ol', 'dl']:
                    paragraphs.append(current_element.get_text(strip=True))
                current_element = current_element.find_next()
            cast_text = '\n'.join(paragraphs)

        return cast_text.strip()

    else:
        print(f"Failed to retrieve the series page. Status code: {response.status_code}")
        return None
df = pd.read_csv('kdramas_info.csv')
df['Series Link'] = df.apply(lambda row: f"https://en.wikipedia.org{row['Drama Link']}" if pd.notnull(row['Drama Link']) and row['Drama Link'].startswith('/wiki/') else None, axis=1)
df['Cast'] = ''
for index, row in df.iterrows():
    series_url = row['Series Link']
    if pd.notnull(series_url):
        cast_info = get_cast_info(series_url)
        if cast_info is not None:
            df.at[index, 'Cast'] = cast_info if cast_info else 'N/A'
df.to_csv('kdramas_info_with_cast.csv', index=False, encoding='utf-8')
print("Kdramas information with cast details saved to 'kdramas_info_with_cast.csv'")


Kdramas information with cast details saved to 'kdramas_info_with_cast.csv'


In [70]:
#adds the synopsis and plot to the culumn and adds it to the casts csv
def get_synopsis_info(series_url):
    response = requests.get(series_url)
    if response.status_code == 200:
        series_soup = BeautifulSoup(response.text, 'html.parser')

        synopsis_heading = series_soup.find('span', {'id': 'Synopsis'})
        plot_heading = series_soup.find('span', {'id': 'Plot'})

        if synopsis_heading:
            parent_section = synopsis_heading.find_parent()
        elif plot_heading:
            parent_section = plot_heading.find_parent()
        else:
            return None

        next_h2 = parent_section.find_next('h2')
        paragraphs = parent_section.find_all_next(['p', 'ul', 'ol', 'dl', 'h2'])
        paragraphs = paragraphs[:paragraphs.index(next_h2)] if next_h2 else paragraphs

        synopsis_text = '\n'.join(paragraph.get_text(strip=True) for paragraph in paragraphs)
        return synopsis_text.strip()

    print(f"Failed to retrieve the series page. Status code: {response.status_code}")
    return None

df = pd.read_csv('kdramas_info_with_cast.csv')
df['Synopsis'] = ''
for index, row in df.iterrows():
    series_url = row['Series Link']
    if pd.notnull(series_url):
        synopsis_info = get_synopsis_info(series_url)
        if synopsis_info is not None:
            df.at[index, 'Synopsis'] = synopsis_info if synopsis_info else 'N/A'

df.to_csv('kdramas_info_with_cast_and_synopsis.csv', index=False, encoding='utf-8')
print("Kdramas cast and synopsis information saved to 'kdramas_info_with_cast_and_synopsis.csv'")

Kdramas cast and synopsis information saved to 'kdramas_info_with_cast_and_synopsis.csv'


In [71]:
# table is created to show the available information

df = pd.read_csv('/content/kdramas_info_with_cast_and_synopsis.csv')

# Display information about series, cast, and synopsis
table_data = []
for index, row in df.iterrows():
    series_name = row['Series']
    cast_info = 'Contains data' if pd.notna(row['Cast']) else 'No data collected'
    synopsis_info = 'Contains data' if pd.notna(row['Synopsis']) else 'No data collected'
    table_data.append([series_name, cast_info, synopsis_info])

headers = ['Series', 'Cast', 'Synopsis']
table = tabulate(table_data, headers=headers, tablefmt='grid')

print("\nK-Dramas Information Table:")
print(table)



K-Dramas Information Table:
+------------------------------------+---------------+---------------+
| Series                             | Cast          | Synopsis      |
+====================================+===============+===============+
| The World of the Married           | Contains data | Contains data |
+------------------------------------+---------------+---------------+
| Reborn Rich                        | Contains data | Contains data |
+------------------------------------+---------------+---------------+
| Sky Castle                         | Contains data | Contains data |
+------------------------------------+---------------+---------------+
| Crash Landing on You               | Contains data | Contains data |
+------------------------------------+---------------+---------------+
| Reply 1988                         | Contains data | Contains data |
+------------------------------------+---------------+---------------+
| Guardian: The Lonely and Great God | Contains 

In [25]:
# Useful and necessary imports
!pip install pandas transformers
import pandas as pd
from transformers import pipeline, BertTokenizerFast, BartForConditionalGeneration
from concurrent.futures import ThreadPoolExecutor
from transformers import pipeline, BertTokenizerFast, BartTokenizerFast

import random
import nltk
from nltk.corpus import stopwords
import time

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Initialize NLP models
qa_model = "bert-large-uncased-whole-word-masking-finetuned-squad"
nlp_bert = pipeline('question-answering', model=qa_model, tokenizer=BertTokenizerFast.from_pretrained('bert-base-uncased'))

# Load the dataset
df = pd.read_csv('/content/kdramas_info_with_cast_and_synopsis.csv')

# Summarization model
summarization_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

from transformers import BartTokenizerFast

def summarize_text(text, max_length=50):
    tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-large-cnn')
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = summarization_model.generate(
        inputs['input_ids'],
        max_length=max_length,
        min_length=40,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-large-cnn')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:


# Initialize scores and excluded dramas list
title_score = 0
actor_score = 0
question_score = 0
excluded_dramas = []

# Welcome message
print("\n📺 Welcome to the Fun K-Drama Guessing Game! 🌟")
print("Think of a K-drama, and I'll try to guess it. Let's get started!")

# Main game loop
while True:
    # User input for action
    user_input = input("\n🔍 Provide a question ('question'), remember the actor ('actor'), or title ('title'), or type 'quit' to exit: ").lower()

    # Exit the game
    if user_input == 'quit':
        print("\n🤖 Chatbot: Thanks for playing! See you next time! 🌟")
        break

    # Question-based guess
    if user_input == 'question':
        user_question = input("\n❓ What specific question would you like to ask about the drama? ")
        print("\n🤖 Chatbot: Let me think about that...")
        time.sleep(2)
        result = nlp_bert(question=user_question, context=df['Synopsis'].str.cat(sep='\n'))

        if 'index' in result:
            relevant_kdrama_details = df.iloc[result['index']]
            series_name = relevant_kdrama_details['Series']
            print(f"\n🎭 Title: {series_name}")
            print(f"📝 Answer: {result['answer']}")
            response = input("\n🤖 Chatbot: Was my guess correct? (Type 'yay' for yes or 'nay' for no): ").lower()

            if response == 'yay':
                print("\n😄 Chatbot: Yay! You earned 10 points!")
                question_score += 10
            elif response == 'nay':
                print("\n🤔 Chatbot: Oops! Let's keep the fun going!")
            else:
                print("\n❗ Chatbot: Please type 'yay' for yes or 'nay' for no.")

        else:
            print("\n❌ Unable to find a relevant Kdrama based on the provided question.")
            print("\n🤖 Chatbot: No worries! Let's keep the fun going!")
            print("🤖 By my Language processing model, this is what you are looking for:", result)

    # Title-based guess
    elif user_input == 'title':
        user_input_title = input("\n📖 Enter the K-drama title: ")
        matching_rows_title = df[df['Series'].str.lower().fillna('').str.contains(user_input_title.lower())]

        if not matching_rows_title.empty:
            best_drama_title = matching_rows_title.iloc[0]['Series']
            print(f"\n🌟 Best Guess: {best_drama_title}")
            # Use summarization model for the synopsis
            summarized_synopsis = summarize_text(matching_rows_title.iloc[0]['Synopsis'])
            print(f"📄 Summary: {summarized_synopsis}")
            title_score += 10
        else:
            print(f"\n🤔 Chatbot: Couldn't find information for the K-drama '{user_input_title}'. Please try again.")
            print("\n😅 Chatbot: No worries! Let's keep the excitement alive!")
            continue

        response = input(f"\n🤖 Chatbot: Is it '{best_drama_title}'? (Type 'yay' for yes, 'nay' for no, or 'ask' for more): ").lower()

        if response == 'yay':
            print("\n🎉 Chatbot: Yay! You're a K-Drama genius! Thanks for playing! 🎊")
            print(f"\n🌟 Your Total Title Score: {title_score}")
            print(f"🌟 Your Total Actor Score: {actor_score}")
            print(f"🌟 Your Total Question Score: {question_score}")
        elif response == 'nay':
            print(f"\n🤔 Chatbot: Okay, let's try another word. Excluding '{best_drama_title}' for now.")
            excluded_dramas.append(best_drama_title)
        elif response == 'ask':
            questions = [input("\n❓ What specific question would you like to ask about the drama? ")]

            with ThreadPoolExecutor() as executor:
                results = list(executor.map(lambda q: nlp_bert(question=q, context=df['Synopsis'].str.cat(sep='\n')), questions))

            for result in results:
                if 'index' in result:
                    relevant_kdrama_details = df.iloc[result['index']]
                    print(f"\n🎭 Title: {relevant_kdrama_details['Series']}")
                    print(f"📝 Answer: {result['answer']}")
                    print("\n😄 Chatbot: Great question! You earned 10 points!")
                    question_score += 10
                else:
                    print("\n❌ Unable to find a relevant Kdrama based on the provided question.")
                    print("\n😅 Chatbot: No problem! Let's continue the fun journey!")

            print("🤖 By my Language processing model, this is what you are looking for:", result)
        else:
            print("\n❗ Chatbot: Please type 'yay' for yes, 'nay' for no, 'ask' for more, or 'quit' to exit.")

    # Actor-based guess
    elif user_input == 'actor':
        user_input_actor = input("\n🕵️ Enter the actor's name: ")
        matching_rows_actor = df[df['Cast'].str.lower().fillna('').str.contains(user_input_actor.lower())]

        if not matching_rows_actor.empty:
            best_drama_actor = matching_rows_actor.iloc[0]['Series']
            print(f"\n🌟 Best Guess: {best_drama_actor}")
            response_actor = input("\n🤖 Chatbot: Was my guess correct? (Type 'yay' for yes or 'nay' for no): ").lower()

            if response_actor == 'yay':
                print("\n😄 Chatbot: Yay! You earned 10 points!")
                actor_score += 10
            elif response_actor == 'nay':
                print("\n🤔 Chatbot: Oops! Let's keep the fun going!")
            else:
                print("\n❗ Chatbot: Please type 'yay' for yes or 'nay' for no.")

        else:
            print(f"\n🤔 Chatbot: Couldn't find a K-drama for the actor '{user_input_actor}'. Please try again.")
            print("\n😅 Chatbot: Don't worry! Let's keep the fun rolling!")

    else:
        print("\n❗ Chatbot: Please type 'question', 'title', 'actor', or 'quit'.")

# Display a closing K-Drama joke
jokes = [
    "Why did the K-Drama cross the road? To reach the next season!",
    "What do you call a K-Drama that makes you laugh and cry? A masterPIECE!",
    "Why did the K-Drama break up with reality TV? It needed more plot twists!"
]
print(f"\n🤖 Chatbot: Thanks for playing! Here's a K-Drama joke for you:\n{random.choice(jokes)}")



📺 Welcome to the Fun K-Drama Guessing Game! 🌟
Think of a K-drama, and I'll try to guess it. Let's get started!

🔍 Provide a question ('question'), remember the actor ('actor'), or title ('title'), or type 'quit' to exit: title

📖 Enter the K-drama title: uncanny

🌟 Best Guess: The Uncanny Counter
📄 Summary: In the fictional city of Jungjin, a group of four demon-hunters called theCountersbear the arduous task of searching for andbanishingevil spirits. The Counters were once incomaswhen a partner spirit from Yung, the boundary between the afterlife and the world of the living,possessesthem and gives them perfectly healthy bodies and consciousness.

🤖 Chatbot: Is it 'The Uncanny Counter'? (Type 'yay' for yes, 'nay' for no, or 'ask' for more): yay

🎉 Chatbot: Yay! You're a K-Drama genius! Thanks for playing! 🎊

🌟 Your Total Title Score: 10
🌟 Your Total Actor Score: 0
🌟 Your Total Question Score: 0

🔍 Provide a question ('question'), remember the actor ('actor'), or title ('title'), or ty

In [4]:
import pandas as pd
import spacy
from concurrent.futures import ThreadPoolExecutor

# Load your kdramas dataset
df = pd.read_csv('/content/kdramas_info_with_cast_and_synopsis.csv')
excluded_dramas = {}
cumulative_scores = {}

# Load spaCy model for named entity recognition
nlp = spacy.load("en_core_web_sm")

# Initialize scores
title_score = 0
actor_score = 0
question_score = 0

# Welcome message
print("\n📺 Welcome to the Fun K-Drama Guessing Game! 🌟")
print("Think of a K-drama, and I'll try to guess it. Let's get started!")

# Main game loop
while True:
    # User input for action
    user_input = input("\n🔍 Provide a keyword ('keyword'), remember the actor ('actor'), title ('title'), or type 'quit' to exit: ").lower()

    # Exit the game
    if user_input == 'quit':
        print("\n🤖 Chatbot: Thanks for playing! See you next time! 🌟")
        break

    # Question-based guess
    if user_input == 'keyword':
        user_question = input("\n❓ What specific kwywords come to mindnay? ")
        print("\n🤖 Chatbot: Let me think about that...")

        # Use spaCy for named entity recognition
        doc = nlp(user_question)
        cast_names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

        df['combined_info'] = df['Series'] + ' ' + df['Synopsis'] + ' ' + df['Cast']

        matching_rows = df[~df['Series'].isin(excluded_dramas.keys())].copy()

        if not matching_rows.empty:
            matching_rows['score'] = matching_rows.apply(
                lambda x: sum(
                    (10 if word.lower() in str(x['Series']).lower() else 1) +
                    (5 if word.lower() in str(x['Synopsis']).lower() else 1) +
                    (2 if any(cast.lower() in str(x['Cast']).lower() for cast in cast_names) else 1)
                    for word in user_question.split()
                )
                if pd.notna(x['Series']) else 0,
                axis=1
            )
            for idx, row in matching_rows.iterrows():
                series = row['Series']
                score = row['score']
                cumulative_scores[series] = cumulative_scores.get(series, 0) + score
            best_drama, highest_score = max(cumulative_scores.items(), key=lambda x: x[1])
            print(f"\nBest Guess: {best_drama} with a cumulative score of {highest_score}")

            relevant_info = matching_rows.loc[matching_rows['score'].idxmax()]
            title = relevant_info['Series']
            response = input(f"\nChatbot: Is it '{title}'? (Type 'yay' for yes, 'nay' for no, or 'ask' for more): ").lower()

            if response == 'yay':
                print("\n😄 Chatbot: Yay! You're correct!")
                question_score += 10
                print(f"\n🌟 Your Total Title Score: {title_score}")
                print(f"🌟 Your Total Actor Score: {actor_score}")
                print(f"🌟 Your Total Question Score: {question_score}")
            elif response == 'nay':
                print(f"\nChatbot: Okay, let's try another word. Excluding '{title}' for now.")
                excluded_dramas[title] = True
            elif response == 'ask':
                # Ask a question to narrow down the possibilities using a simple keyword search
                user_question = input("\nChatbot: What specific question would you like to ask about the drama? ")
                relevant_rows = df[df['Synopsis'].str.contains(user_question, case=False)]

                if not relevant_rows.empty:
                    print("\nChatbot: Here are some dramas that match your question:")
                    print(relevant_rows[['Series', 'Synopsis']])
                else:
                    print("Chatbot: I couldn't find relevant information based on your question.")
            else:
                print("\nChatbot: Please type 'yay' for yes, 'nay' for no, or 'ask' for more.")
        else:
            print("\nChatbot: I'm sorry, I couldn't find relevant information based on your input.")

    # Title-based guess
    elif user_input == 'title':
        user_input_title = input("\n📖 Enter the K-drama title: ")
        matching_rows_title = df[df['Series'].str.lower().fillna('').str.contains(user_input_title.lower())]

        if not matching_rows_title.empty:
            best_drama_title = matching_rows_title.iloc[0]['Series']
            print(f"\n🌟 Best Guess: {best_drama_title}")
            # Use summarization model for the synopsis
            summarized_synopsis = summarize_text(matching_rows_title.iloc[0]['Synopsis'])
            print(f"📄 Summary: {summarized_synopsis}")
            title_score += 10
        else:
            print(f"\n🤔 Chatbot: Couldn't find information for the K-drama '{user_input_title}'. Please try again.")
            print("\n😅 Chatbot: No worries! Let's keep the excitement alive!")
            continue

        response = input(f"\n🤖 Chatbot: Is it '{best_drama_title}'? (Type 'yay' for yes, 'nay' for no, or 'ask' for more): ").lower()

        if response == 'yay':
            print("\n🎉 Chatbot: Yay! You're a K-Drama genius!")
            title_score += 10
            print(f"\n🌟 Your Total Title Score: {title_score}")
            print(f"🌟 Your Total Actor Score: {actor_score}")
            print(f"🌟 Your Total Question Score: {question_score}")
        elif response == 'nay':
            print(f"\nChatbot: Okay, let's try another word. Excluding '{best_drama_title}' for now.")
            excluded_dramas[best_drama_title] = True
        elif response == 'ask':
            questions = [input("\n❓ What specific question would you like to ask about the drama? ")]

            with ThreadPoolExecutor() as executor:
                results = list(executor.map(lambda q: nlp_bert(question=q, context=df['Synopsis'].str.cat(sep='\n')), questions))

            for result in results:
                if 'index' in result:
                    relevant_kdrama_details = df.iloc[result['index']]
                    print(f"\n🎭 Title: {relevant_kdrama_details['Series']}")
                    print(f"📝 Answer: {result['answer']}")
                    print("\n😄 Chatbot: Great question! You earned 10 points!")
                    question_score += 10
                else:
                    print("\n❌ Unable to find a relevant Kdrama based on the provided question.")
                    print("\n😅 Chatbot: No problem! Let's continue the fun journey!")

                print("🤖 By my Language processing model, this is what you are looking for:", result)
        else:
            print("\n❗ Chatbot: Please type 'yay' for yes, 'nay' for no, 'ask' for more, or 'quit' to exit.")

    # Actor-based guess
    elif user_input == 'actor':
        user_input_actor = input("\n🕵️ Enter the actor's name: ")
        matching_rows_actor = df[df['Cast'].str.lower().fillna('').str.contains(user_input_actor.lower())]

        if not matching_rows_actor.empty:
            best_drama_actor = matching_rows_actor.iloc[0]['Series']
            print(f"\n🌟 Best Guess: {best_drama_actor}")
            response_actor = input("\n🤖 Chatbot: Was my guess correct? (Type 'yay' for yes or 'nay' for no): ").lower()

            if response_actor == 'yay':
                print("\n😄 Chatbot: Yay! You earned 10 points!")
                actor_score += 10
            elif response_actor == 'nay':
                print("\n🤔 Chatbot: Oops! Let's keep the fun going!")
            else:
                print("\n❗ Chatbot: Please type 'yay' for yes or 'nay' for no.")

        else:
            print(f"\n🤔 Chatbot: Couldn't find a K-drama for the actor '{user_input_actor}'. Please try again.")
            print("\n😅 Chatbot: Don't worry! Let's keep the fun rolling!")

    else:
        print("\n❗ Chatbot: Please type 'question', 'title', 'actor', or 'quit'.")



📺 Welcome to the Fun K-Drama Guessing Game! 🌟
Think of a K-drama, and I'll try to guess it. Let's get started!

🔍 Provide a keyword ('keyword'), remember the actor ('actor'), title ('title'), or type 'quit' to exit: keyword

❓ What specific kwywords come to mind? jeju 

🤖 Chatbot: Let me think about that...

Best Guess: Our Blues with a cumulative score of 7


KeyboardInterrupt: ignored